In [1]:
#!/usr/bin python2.7

from astropy.io       import fits
from astropy.table    import Table
from astropy.time     import Time
from datetime         import date
from datetime         import datetime
import ephem
import fileinput
import getpass
import math
import matplotlib.pyplot                   as plt
import matplotlib                          as mpl
import numpy                               as np
from numpy            import linalg        as LA
import os
import pyfits
import scipy.optimize                      as optimization
from scipy            import integrate
from scipy            import interpolate
from scipy.optimize   import curve_fit
import subprocess
from subprocess       import Popen, PIPE
import urllib2
from work_module      import calculate
from work_module      import detector
from work_module      import readfile
from work_module      import writefile
calc = calculate()
det = detector()
rf = readfile()
wf = writefile()

In [2]:
#calculate flux coming from one defined source
detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'na', 'nb'])

#burst_ras = np.array([211.91936, 41.1766, 63.5977, 14.3462, 0.1992, 339.25167])
#burst_decs = np.array([1.33325, -26.1544, -58.8298, -46.8005, 44.6017, 56.88928])

burst_ras = np.array([14.3462])
burst_decs = np.array([-46.8005])

#start_time = np.array([20., 4.09, 6., 6., 17., 8.])
start_time = np.array([6.367])

#end_time = np.array([24.1, 4.29, 18., 18., 24.1, 10.])
end_time = np.array([9.422])

#burst_array = np.array([[21.29361], [4.19], [7.54139], [7.20222], [18.62917], [8.932908]])
burst_array = np.array([[7.20222]])


plot_detectors = np.array(['n0', 'n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9', 'na', 'nb'])
echans = np.array([0, 1, 2, 3, 4, 5, 6, 7])
#detectors = np.array(['n3', 'n5', 'n0', 'n9'])

#days = np.array([140102, 130925, 121027, 111209, 101225, '091024'])
days = np.array([111209])
#burst_ra = 60.68
#burst_dec = 32.77
binning = 1

user = getpass.getuser()

#start_time = 7.9
#end_time = 8.2


#burst_occ = calc.src_occultation(day, burst_ra, burst_dec)[0]

#sat_time = rf.poshist(day)[0]

#plot_time_sat_date = calc.met_to_date(sat_time)[0]
#plot_time_sat = (plot_time_sat_date - calc.day_to_met(day)[1])*24#Time of day in hours

#mpl.rcParams['axes.color_cycle'] = ['g', 'r', 'c', 'm', 'y', 'k']

for j in range(0, len(days)):
    day = days[j]
    print day
    
    year = int('20' + str(day)[0:2])
            
    #get the iso-date-format from the day
    date = datetime(year, int(str(day)[2:4]), int(str(day)[4:6]))
            
    #get the ordinal indicator for the date
    ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])
    
    burst_ra = burst_ras[j]
    burst_dec = burst_decs[j]
    
    burst_occ = calc.src_occultation(day, burst_ra, burst_dec)[0]
    
    #read the SFL data
    flares = readfile.flares(readfile(), year)
    flares_day = flares[0]
    flares_time = flares[1]
    flares_today = flares_time[:,np.where(flares_day == int(day))]
    flares_today = np.squeeze(flares_today, axis=(1,))/3600.
    
    mpl.pyplot.rcParams['agg.path.chunksize'] = 20000

    for echan in echans:
        print echan
        
        detec_residuals = []
        detec_counts = []
        detec_ang = []
            
        for i in range(0, len(detectors)):
                
            ctime_data = calc.curve_fit_plots(day, detectors[i], echan)
            det_residuals = ctime_data[0]
            det_counts = ctime_data[1]
            det_fit = ctime_data[2]
            plot_time_bin = ctime_data[8]
            plot_time_sat = ctime_data[9]
                
            #eff_residuals = det_residuals*det_ang_eff
            #eff_counts = det_counts*det_ang_eff
                
            binned_res = det_residuals[0::binning]
            binned_counts = det_counts[0::binning]
            binned_times_bin = plot_time_bin[int(binning/2)::binning]
            binned_times_sat = plot_time_sat[int(binning/2)::binning]
    
            for j in range(1, binning):
                if len(binned_res) != len(det_residuals[j::binning]):
                    binned_res = np.delete(binned_res, -1)
                    binned_counts = np.delete(binned_counts, -1)
                binned_res = np.add(binned_res, det_residuals[j::binning])
                binned_counts = np.add(binned_counts, det_counts[j::binning])
            
            if len(binned_res) < len(binned_times_bin):
                binned_times_bin = np.delete(binned_times_bin, -1)
                binned_times_sat = np.delete(binned_times_sat, -1)
                
            detec_residuals.append(binned_res)
            detec_counts.append(binned_counts)
            
            det_data = calc.burst_ang_bin(detectors[i], day, burst_ra, burst_dec)
            det_ang = det_data[0]
            detec_ang.append(det_ang)
        
        
            #det_ang[np.where(det_ang > 90.)] = 0.
            
            #plt.plot(binned_times_bin, binned_res, label = detectors[i])
                
                
                
        detec_residuals = np.array(detec_residuals)/binning
        detec_counts = np.array(detec_counts)/binning
        detec_ang = np.array(detec_counts)
        #plot_residuals = np.squeeze(np.sum(plot_residuals, axis = 0))
        #plot_counts = np.squeeze(np.sum(plot_counts, axis = 0))
                
        #print plot_time_bin
        #print tot_residuals
        
        effective_residuals = []
        
        for plot_detector in plot_detectors:
            print '   ', plot_detector
    
            if plot_detector == 'n0':
                index = 0
                plot_residuals = detec_residuals[0] - 0.5*detec_residuals[8] - 0.5*detec_residuals[11]
            elif plot_detector == 'n1':
                index = 1
                plot_residuals = detec_residuals[1] - 0.5*detec_residuals[8] - 0.5*detec_residuals[11]
            elif plot_detector == 'n2':
                index = 2
                plot_residuals = detec_residuals[2] - detec_residuals[8]
            elif plot_detector == 'n3':
                index = 3
                plot_residuals = detec_residuals[3] - 0.5*detec_residuals[10] - 0.5*detec_residuals[11]
            elif plot_detector == 'n4':
                index = 4
                plot_residuals = detec_residuals[4] - detec_residuals[10]
            elif plot_detector == 'n5':
                index = 5
                plot_residuals = detec_residuals[5] - detec_residuals[11]
            elif plot_detector == 'n6':
                index = 6
                plot_residuals = detec_residuals[6] - 0.5*detec_residuals[2] - 0.5*detec_residuals[5]
            elif plot_detector == 'n7':
                index = 7
                plot_residuals = detec_residuals[7] - 0.5*detec_residuals[2] - 0.5*detec_residuals[5]
            elif plot_detector == 'n8':
                index = 8
                plot_residuals = detec_residuals[8] - detec_residuals[2]
            elif plot_detector == 'n9':
                index = 9
                plot_residuals = detec_residuals[9] - 0.5*detec_residuals[4] - 0.5*detec_residuals[5]
            elif plot_detector == 'na':
                index = 10
                plot_residuals = detec_residuals[10] - detec_residuals[4]
            elif plot_detector == 'nb':
                index = 11
                plot_residuals = detec_residuals[11] - detec_residuals[5]
                
            
            effective_residuals.append(plot_residuals)
        effective_residuals = np.array(effective_residuals)
        
        
        index = np.argmin(detec_ang, axis = 0)
        
        source_curve = []
        for k in range(0, len(detec_ang[0])):
            source_curve.append(effective_residuals[index[k]][k])
        source_curve = np.array(source_curve)
        
        plt.axvline(x = burst_array[j], ymin = 0., ymax = 1., linewidth=2, color = 'black', linestyle = '--')
        
        if flares_today.size != 0:
            if len(flares_today[0]) > 1:
                for i in range(0, len(flares_today[0])):
                    plt.axvline(x = flares_today[0,i], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
                    plt.axvline(x = flares_today[1,i], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')
            else:
                plt.axvline(x = flares_today[0], ymin = 0., ymax = 1., linewidth=2, color = 'grey')
                plt.axvline(x = flares_today[1], ymin = 0., ymax = 1., color = 'grey', linestyle = '--')
        
        plt.step(binned_times_bin, source_curve, 'b-', label = 'Adjusted effective residuals')
        plt.grid()
        plt.legend(fontsize=30)
                
        #plt.xlim([-0.5, 24.5])
        plt.xlim([start_time[j], end_time[j]])
        plt.ylim([-300, 300])
                
        plt.xlabel('Time of day in 24h', fontsize=40)
        plt.ylabel('Residual noise', fontsize=40)
        plt.tick_params(axis='both', which='major', labelsize=30)
                
        plt.title('Adjusted effective residuals of the ULGRB-source on the ' + ordinal(int(str(day)[4:6])) + ' ' + date.strftime('%B')[0:3] + ' ' + str(year), fontsize=45)
                
                
        figure_name = str(day) + '_src_eff_e' + str(echan) + '.png'
           
           
        fig = plt.gcf() # get current figure
        fig.set_size_inches(20, 12)
        
        figure_path = '/home/' + user + '/Work/Fits/' + str(day) + '/'
        figure = os.path.join(figure_path, figure_name)
        plt.savefig(figure, bbox_inches='tight', dpi = 80)
                
        #plt.show()

        fig.clf()
        plt.clf()

111209
0
    n0
    n1
    n2
    n3
    n4
    n5
    n6
    n7
    n8
    n9
    na
    nb
1
    n0
    n1
    n2
    n3
    n4
    n5
    n6
    n7
    n8
    n9
    na
    nb
2
    n0
    n1
    n2
    n3
    n4
    n5
    n6
    n7
    n8
    n9
    na
    nb
3
    n0
    n1
    n2
    n3
    n4
    n5
    n6
    n7
    n8
    n9
    na
    nb
4
    n0
    n1
    n2
    n3
    n4
    n5
    n6
    n7
    n8
    n9
    na
    nb
5
    n0
    n1
    n2
    n3
    n4
    n5
    n6
    n7
    n8
    n9
    na
    nb
6
    n0
    n1
    n2
    n3
    n4
    n5
    n6
    n7
    n8
    n9
    na
    nb
7
    n0
    n1
    n2
    n3
    n4
    n5
    n6
    n7
    n8
    n9
    na
    nb
